In [ ]:
# In: notebooks/03_Feature_Engineering_Splitting.ipynb
# Purpose: Load the defined cohort, engineer time-based features,
#          select features (incl. pre-computed MRI metrics) for baseline modeling,
#          perform subject-level stratified split, and save the split datasets.

In [ ]:
# --- Import Libraries ---
import pandas as pd
import numpy as np
import wandb
import json
from pathlib import Path
import time
import os
from sklearn.model_selection import train_test_split

In [ ]:
# --- Config Loading ---
print("--- Loading Configuration ---")
CONFIG_PATH = Path('../config.json') # Path relative to the notebook location
try:
    PROJECT_ROOT = CONFIG_PATH.parent.resolve()
    print(f"Project Root detected as: {PROJECT_ROOT}")

    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        config = json.load(f)
    print("Configuration loaded successfully.")

    # Define key variables from config
    OUTPUT_DIR_BASE = PROJECT_ROOT / config['data']['output_dir_base']
    WANDB_PROJECT = config['wandb']['project_name']
    WANDB_ENTITY = config['wandb'].get('entity', None)

    # Define input path (output from Notebook 02)
    NB02_OUTPUT_DIR = OUTPUT_DIR_BASE / "02_Cohort_Definition"
    COHORT_CSV_PATH = NB02_OUTPUT_DIR / "final_analysis_cohort.csv" # Output from NB 02

    # Define specific output dir for this notebook's results and create it
    NOTEBOOK_NAME = "03_Feature_Engineering_Splitting"
    output_dir = OUTPUT_DIR_BASE / NOTEBOOK_NAME
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Outputs will be saved to: {output_dir}")

except Exception as e:
    print(f"Error loading config or setting up paths: {e}")
    exit()

# --- Initialize W&B Run ---
print("\n--- Initializing Weights & Biases Run ---")
run = None # Initialize run to None
try:
    run = wandb.init(
        project=WANDB_PROJECT,
        entity=WANDB_ENTITY,
        job_type="feature-engineering-splitting",
        name=f"{NOTEBOOK_NAME}-run-{time.strftime('%Y%m%d-%H%M')}",
        config={ # Log key config choices for this job
            "input_cohort_artifact": f"analysis_cohort-OASIS2...", # Ideally get full name from NB02 run
            "input_cohort_path": str(COHORT_CSV_PATH),
            "time_feature_source": "'MR Delay' preferred, fallback 'Age'", # Documenting strategy
            # Split ratios etc. here later via wandb.config.update()
        }
    )
    print(f"W&B run '{run.name}' initialized successfully. View at: {run.url}")
except Exception as e:
    print(f"Error initializing W&B: {e}")
    print("Proceeding without W&B logging.")

## Load Defined Cohort Data

Load the `final_analysis_cohort.csv` file generated by Notebook 02. This contains the subjects and visits meeting our baseline CDR, minimum visit, and MRI availability criteria.

In [ ]:
print(f"\n--- Loading Defined Cohort Data from: {COHORT_CSV_PATH} ---")
try:
    if not COHORT_CSV_PATH.is_file():
         raise FileNotFoundError(f"Cohort data file not found at {COHORT_CSV_PATH}. Ensure Notebook 02 ran successfully.")
    cohort_df = pd.read_csv(COHORT_CSV_PATH)
    print(f"Cohort data loaded successfully. Shape: {cohort_df.shape}")
    if run:
        run.log({'feature_engineering/input_cohort_rows': len(cohort_df),
                 'feature_engineering/input_cohort_subjects': cohort_df['Subject ID'].nunique()})
    print("\nCohort DataFrame Head:")
    print(cohort_df.head())
    print("\nCohort DataFrame Info:")
    cohort_df.info()

except FileNotFoundError as e:
    print(f"Error: {e}")
    if run: run.finish()
    exit()
except Exception as e:
    print(f"An error occurred loading the cohort data: {e}")
    if run: run.finish()
    exit()

## Engineer Time Features

Calculate time relative to each subject's baseline visit within this cohort and the time elapsed since the previous visit. We prioritize using the `MR Delay` column if available (assuming it's days from the first visit), otherwise, we approximate using the `Age` column.

In [ ]:
print("\n--- Engineering Time-Based Features ---")

# Ensure data is sorted for time calculations
cohort_df = cohort_df.sort_values(by=['Subject ID', 'Visit']).copy()

# Prioritize 'MR Delay' if it exists and seems correct (days from baseline visit 1)
if 'MR Delay' in cohort_df.columns and cohort_df['MR Delay'].isnull().sum() < len(cohort_df) * 0.5 : # Check if MR Delay mostly exists
    print("Using 'MR Delay' to calculate time features.")
    # Ensure MR Delay is numeric
    cohort_df['MR Delay'] = pd.to_numeric(cohort_df['MR Delay'], errors='coerce')
    # Find the MR Delay of the *first included visit* in our cohort for each subject
    cohort_df['BaselineVisitMRDelay'] = cohort_df.groupby('Subject ID')['MR Delay'].transform('min')
    # Calculate Days_from_Baseline relative to the first visit *in the cohort*
    cohort_df['Days_from_Baseline'] = cohort_df['MR Delay'] - cohort_df['BaselineVisitMRDelay']
    # Calculate time since last visit using the diff on Days_from_Baseline
    cohort_df['Time_since_Last_Visit_Days'] = cohort_df.groupby('Subject ID')['Days_from_Baseline'].diff()
    time_feature_source = 'MR Delay'

elif 'Age' in cohort_df.columns:
    print("Warning: Using 'Age' column to approximate time features. Check if 'MR Delay' exists and is suitable.")
    # Calculate Days_from_Baseline based on age difference from first visit age
    cohort_df['BaselineAge'] = cohort_df.groupby('Subject ID')['Age'].transform('min')
    cohort_df['Days_from_Baseline'] = (cohort_df['Age'] - cohort_df['BaselineAge']) * 365.25
    # Calculate time since last visit using the diff on Age
    cohort_df['Time_since_Last_Visit_Days'] = cohort_df.groupby('Subject ID')['Age'].diff() * 365.25
    time_feature_source = 'Age_approx'

else:
    print("Error: Neither 'MR Delay' nor 'Age' suitable for calculating time features.")
    if run: run.finish()
    exit()

# Fill NaN for the first visit of each subject with 0
cohort_df['Time_since_Last_Visit_Days'].fillna(0, inplace=False) # Use assignment, not inplace
# Replace the above line with:
cohort_df['Time_since_Last_Visit_Days'] = cohort_df['Time_since_Last_Visit_Days'].fillna(0)

print("Calculated 'Days_from_Baseline' and 'Time_since_Last_Visit_Days'.")

# Display results for verification
print("\nExample Time Features:")
print(cohort_df[['Subject ID', 'Visit', 'Age', 'MR Delay' if 'MR Delay' in cohort_df.columns else 'Age' , 'Days_from_Baseline', 'Time_since_Last_Visit_Days']].head())

# Basic check on calculated time features
print("\nTime Feature Statistics:")
desc_time_features = cohort_df[['Days_from_Baseline', 'Time_since_Last_Visit_Days']].describe()
print(desc_time_features)
if run:
    run.config.update({'feature_engineering/time_feature_source': time_feature_source})
    for col in desc_time_features.columns:
        for idx in desc_time_features.index:
             run.log({f'stats/time_features/{col}_{idx}': desc_time_features.loc[idx, col]})

## Prepare Static Features and Select Final Columns

Extract baseline features (CDR, MMSE) for each subject to use as static inputs. Select the final set of columns needed for the modeling dataset, including identifiers, time-varying features (clinical + time + pre-computed MRI like nWBV), static features, and the base target variable ('CDR').

In [ ]:
print("\n--- Preparing Static Features & Selecting Final Columns ---")

# Extract Baseline CDR and MMSE using transform('first') on the sorted dataframe
cohort_df['Baseline_CDR'] = cohort_df.groupby('Subject ID')['CDR'].transform('first')
if 'MMSE' in cohort_df.columns:
    cohort_df['Baseline_MMSE'] = cohort_df.groupby('Subject ID')['MMSE'].transform('first')
    print("Extracted Baseline CDR and Baseline MMSE.")
    # Handle potential NaNs if the first visit MMSE was missing
    cohort_df['Baseline_MMSE'] = cohort_df.groupby('Subject ID')['Baseline_MMSE'].ffill().bfill() # Simple forward/backward fill within subject
else:
    cohort_df['Baseline_MMSE'] = np.nan
    print("Extracted Baseline CDR. MMSE column not found or not used.")

# Define columns to keep
# Focus on features available in the cohort_df for the *initial* model (using nWBV etc.)
time_varying_features = [
    'Age',      # Age at current visit
    'MMSE',     # MMSE at current visit (handle missing via imputation later)
    'nWBV',     # Pre-computed MRI feature (handle missing via imputation later)
    'Days_from_Baseline',         # Engineered time feature
    'Time_since_Last_Visit_Days'  # Engineered time feature
]
static_features = [
    'M/F',      # Sex (needs encoding later)
    'EDUC',     # Education (handle missing via imputation later?)
    'SES',      # SES (handle missing via imputation later)
    'Baseline_CDR', # Extracted baseline feature
    'Baseline_MMSE',# Extracted baseline feature (handle missing via imputation later)
    'eTIV',     # Often static per subject, treat as static? Or check variability. Let's assume static for now.
    'ASF',      # Often static per subject, treat as static? Let's assume static for now.
]
identifiers = ['Subject ID', 'Visit', 'MRI ID'] # Keep MRI ID for potential future use/linking
target_base = ['CDR'] # Base column for creating the target prediction

# Filter lists based on actual columns present
available_columns = cohort_df.columns.tolist()
time_varying_features = [f for f in time_varying_features if f in available_columns]
static_features = [f for f in static_features if f in available_columns]
identifiers = [f for f in identifiers if f in available_columns]
target_base = [f for f in target_base if f in available_columns]

# Check if essential columns are present
if 'Subject ID' not in identifiers or 'Visit' not in identifiers or not target_base:
     print("Error: Essential identifier or target columns ('Subject ID', 'Visit', 'CDR') are missing!")
     if run: run.finish()
     exit()

columns_to_keep = sorted(list(set(identifiers + time_varying_features + static_features + target_base)))

print(f"\nSelected features for modeling dataset:")
print(columns_to_keep)

# Create the feature DataFrame
feature_df = cohort_df[columns_to_keep].copy()
print(f"\nShape of feature DataFrame before target creation: {feature_df.shape}")

# Log selected features
if run:
    wandb.config.update({
        "features/identifiers": identifiers,
        "features/time_varying": time_varying_features,
        "features/static": static_features,
        "features/target_base": target_base
        })

## Create Target Variable (Next Visit CDR)

Generate the target variable for our sequence prediction task. This will be the CDR score from the *next* available visit for each subject. Visits that are the last recorded visit for a subject will not have a target and will be removed.

In [ ]:
print("\n--- Creating Target Variable (CDR at Next Visit) ---")

# Ensure sorting before shifting
feature_df = feature_df.sort_values(by=['Subject ID', 'Visit']).copy()

# Shift CDR scores up within each subject group to get the next visit's score
feature_df['CDR_next_visit'] = feature_df.groupby('Subject ID')['CDR'].shift(-1)

# Handle the last visit for each subject - they will have NaN for the target
initial_rows_target = len(feature_df)
feature_df.dropna(subset=['CDR_next_visit'], inplace=True)
rows_after_target_dropna = len(feature_df)
rows_dropped_last_visit = initial_rows_target - rows_after_target_dropna

print(f"Removed {rows_dropped_last_visit} rows corresponding to the last visit of each subject (no future CDR available).")
print(f"Shape of final modeling DataFrame: {feature_df.shape}")

if run:
    run.log({
        'feature_engineering/rows_before_target_dropna': initial_rows_target,
        'feature_engineering/rows_dropped_for_last_visit': rows_dropped_last_visit,
        'feature_engineering/final_rows_with_target': rows_after_target_dropna
    })

if feature_df.empty:
    print("Error: No data remaining after creating target variable. Check cohort definition and data.")
    if run: run.finish()
    exit()


In [ ]:
# --- Create Target Variable (Next Visit CDR) ---
print("\n--- Creating Target Variable (CDR at Next Visit) ---")

# Shift CDR scores up within each subject group to get the next visit's score
feature_df['CDR_next_visit'] = feature_df.groupby('Subject ID')['CDR'].shift(-1)

# Handle the last visit for each subject - they will have NaN for the target
initial_rows = len(feature_df)
feature_df.dropna(subset=['CDR_next_visit'], inplace=True)
rows_after_target_dropna = len(feature_df)
rows_dropped_last_visit = initial_rows - rows_after_target_dropna

print(f"Removed {rows_dropped_last_visit} rows corresponding to the last visit of each subject (no future CDR available).")
print(f"Shape of final modeling DataFrame: {feature_df.shape}")

if run:
    run.log({
        'feature_engineering/rows_before_target_dropna': initial_rows,
        'feature_engineering/rows_dropped_for_last_visit': rows_dropped_last_visit,
        'feature_engineering/final_rows_with_target': rows_after_target_dropna
    })

if feature_df.empty:
    print("Error: No data remaining after creating target variable. Check cohort definition and data.")
    if run: run.finish()
    exit()

## Perform Subject-Level Stratified Train/Validation/Test Split

Split the data into training, validation, and test sets. This split MUST be done at the **subject level** to prevent data leakage (i.e., all visits from one subject belong to the same set). We will stratify the split based on the **Baseline CDR** (0 vs 0.5) to ensure similar group representation across sets.

In [ ]:
print("\n--- Performing Stratified Train/Validation/Test Split by Subject ---")

# Get unique subjects and their baseline CDR for stratification
# Ensure we take the value associated with the MINIMUM visit per subject in this df
subject_baseline_cdr = feature_df.loc[feature_df.groupby('Subject ID')['Visit'].idxmin()][['Subject ID', 'Baseline_CDR']].copy()
subject_baseline_cdr.set_index('Subject ID', inplace=True)

unique_subjects = subject_baseline_cdr.index.unique()
subject_labels = subject_baseline_cdr['Baseline_CDR']

if len(unique_subjects) < 50: # Check if cohort size is very small
    print(f"Warning: Very small number of subjects ({len(unique_subjects)}) for splitting.")

# Define split ratios (ensure they sum to <= 1)
TEST_SET_RATIO = 0.15
VAL_SET_RATIO = 0.15 # Proportion of the *original* data for validation
RANDOM_STATE = 42
# Set a fixed random state (integer) for the train/test splitting functions.
# Using the same state ensures that the *exact same subjects* are assigned
# to the train, validation, and test sets every time this code is executed.
# This is essential for reproducible results and fair comparison between experiments.

# Calculate relative validation size needed for the second split
if (1.0 - TEST_SET_RATIO) <= 0:
     print("Error: Test set ratio must be less than 1.")
     exit()
relative_val_size = VAL_SET_RATIO / (1.0 - TEST_SET_RATIO)

print(f"Splitting {len(unique_subjects)} subjects.")
print(f"Target split ratios -> Test: {TEST_SET_RATIO:.1%}, Validation: {VAL_SET_RATIO:.1%}, Train: {1.0 - TEST_SET_RATIO - VAL_SET_RATIO:.1%}")

# Split subjects into Train and Temp (Val+Test)
try:
    train_subjects, temp_subjects, train_labels, temp_labels = train_test_split(
        unique_subjects, subject_labels,
        test_size=TEST_SET_RATIO, # Reserve test set first
        random_state=RANDOM_STATE,
        stratify=subject_labels
    )

    # Split Temp into Validation and Test
    # Handle cases where temp set might be too small for stratification
    if len(temp_subjects) < 2 or len(np.unique(temp_labels)) < 2:
         print("Warning: Temp set (Val+Test) too small or lacks diversity for stratification. Performing non-stratified split for Val/Test.")
         # Fallback to non-stratified split if necessary
         if len(temp_subjects) < 2:
              # Handle edge case: put all temp into val? or test? Or error? Let's put into val for now.
              val_subjects = temp_subjects
              test_subjects = np.array([]) # No test subjects
         else:
              val_subjects, test_subjects = train_test_split(
                   temp_subjects,
                   test_size=TEST_SET_RATIO / (TEST_SET_RATIO + VAL_SET_RATIO), # Adjust test size relative to temp set
                   random_state=RANDOM_STATE
              )
    else:
         # Proceed with stratified split
         val_subjects, test_subjects, _, _ = train_test_split(
             temp_subjects, temp_labels,
             test_size=TEST_SET_RATIO / (TEST_SET_RATIO + VAL_SET_RATIO), # Adjust test size relative to temp set
             random_state=RANDOM_STATE,
             stratify=temp_labels
         )

    print(f"\nSplit complete:")
    print(f"  Train subjects: {len(train_subjects)}")
    print(f"  Validation subjects: {len(val_subjects)}")
    print(f"  Test subjects: {len(test_subjects)}")
    print(f"  Total subjects accounted for: {len(train_subjects) + len(val_subjects) + len(test_subjects)}")

    # --- Create the split DataFrames ---
    train_df = feature_df[feature_df['Subject ID'].isin(train_subjects)].copy()
    val_df = feature_df[feature_df['Subject ID'].isin(val_subjects)].copy()
    test_df = feature_df[feature_df['Subject ID'].isin(test_subjects)].copy()

    print(f"\nDataFrames created:")
    print(f"  Train DF shape: {train_df.shape}")
    print(f"  Validation DF shape: {val_df.shape}")
    print(f"  Test DF shape: {test_df.shape}")

    # Log split details
    if run:
        run.log({
            'split/num_subjects_train': len(train_subjects),
            'split/num_subjects_val': len(val_subjects),
            'split/num_subjects_test': len(test_subjects),
            'split/num_visits_train': len(train_df),
            'split/num_visits_val': len(val_df),
            'split/num_visits_test': len(test_df),
        })
        wandb.config.update({
            'split/test_set_ratio': TEST_SET_RATIO,
            'split/validation_set_ratio': VAL_SET_RATIO,
            'split/stratify_by': 'Baseline_CDR',
            'split/random_state': RANDOM_STATE
        })

        # Verify stratification (optional but recommended log)
        for split_name, df_split in [('train', train_df), ('validation', val_df), ('test', test_df)]:
             if not df_split.empty:
                 baseline_dist = df_split.drop_duplicates(subset=['Subject ID'])['Baseline_CDR'].value_counts(normalize=True)
                 for cdr_val, prop in baseline_dist.items():
                     run.log({f'split_check/{split_name}_prop_cdr_{cdr_val}': prop})
        print("Logged split counts and stratification check to W&B.")

except ValueError as e:
    print(f"\nError during stratified split: {e}")
    print("This can happen if a group (e.g., CDR=0.5) has too few members (<= n_splits) for the requested splits.")
    print("Consider adjusting split ratios or reviewing the cohort definition.")
    if run: run.finish()
    exit()
except Exception as e:
     print(f"An unexpected error occurred during splitting: {e}")
     if run: run.finish()
     exit()

## Save Split DataFrames and Log Artifacts

Save the resulting train, validation, and test DataFrames locally as efficient Parquet files. Log these files as versioned artifacts in Weights & Biases for reproducibility and easy access in later modeling stages.

In [ ]:
print("\n--- Saving Split DataFrames and Logging Artifacts ---")

# Ensure we have DataFrames to save, even if empty after split error handling
split_files = {
    "train": train_df if 'train_df' in locals() else pd.DataFrame(),
    "validation": val_df if 'val_df' in locals() else pd.DataFrame(),
    "test": test_df if 'test_df' in locals() else pd.DataFrame()
}
artifact_type = "split-dataframe" # Use a descriptive type

for split_name, df_split in split_files.items():
    if df_split is not None and not df_split.empty:
        # Define local save path
        file_path = output_dir / f"cohort_{split_name}.parquet"
        artifact_name = f"cohort-split-{split_name}" # e.g., cohort-split-train

        try:
            # Save locally
            df_split.to_parquet(file_path, index=False)
            print(f"{split_name.capitalize()} DataFrame saved locally to {file_path} (Shape: {df_split.shape})")

            # Log as W&B artifact
            if run:
                print(f"Logging {split_name} DataFrame as W&B artifact: {artifact_name}...")
                description = (f"{split_name.capitalize()} split of the analysis cohort "
                               f"({df_split['Subject ID'].nunique()} subjects, {len(df_split)} visits). "
                               f"Features engineered, ready for sequence creation/scaling/imputation.")
                artifact = wandb.Artifact(artifact_name, type=artifact_type, description=description,
                                         metadata={'split': split_name,
                                                   'num_rows': len(df_split),
                                                   'num_subjects': df_split['Subject ID'].nunique(),
                                                   'columns': df_split.columns.tolist()})
                artifact.add_file(str(file_path))
                run.log_artifact(artifact)
                print(f"{split_name.capitalize()} artifact logged.")

        except Exception as e:
            print(f"Warning: Could not save or log {split_name} DataFrame. Error: {e}")
    else:
        print(f"Skipping saving/logging for empty or None {split_name} DataFrame.")

## Next Steps

The split DataFrames (`cohort_train.parquet`, `cohort_validation.parquet`, `cohort_test.parquet`) containing engineered and selected features are now saved locally and logged as W&B artifacts.

The subsequent stages involve:

1.  **Data Loading Pipeline:** Create PyTorch/TensorFlow `Dataset` and `DataLoader` classes to:
    * Load data from the saved parquet files.
    * Create sequences of visits for each subject.
    * Handle padding and masking for variable sequence lengths.
    * **Fit Imputers/Scalers:** Fit `SimpleImputer` (for SES, MMSE, nWBV etc. if needed) and `StandardScaler` **only on the training data (`cohort_train.parquet`)**.
    * **Apply Imputation/Scaling:** Transform train, validation, and test data using the *fitted* imputers/scalers.
    * Encode categorical features (e.g., 'M/F').
    * Prepare batches for the model.
2.  **MRI Feature Extraction:** Implement the pipeline to preprocess raw T1w images and extract features using the baseline 3D CNN. This is a separate, significant task that needs to align with the subject/scan IDs in these splits.
3.  **Model Implementation:** Define the sequence model (LSTM/GRU) architecture, incorporating the combined clinical/demographic/time/MRI features and dropout.
4.  **Training & Evaluation:** Implement the training loop, loss function, optimizer, evaluation metrics, and MC Dropout for uncertainty.

In [ ]:
# --- Finish W&B Run ---
print("\n--- Feature Engineering and Splitting complete. Finishing W&B run. ---")
if run:
    run.finish()
    print("W&B run finished.")
else:
    print("No active W&B run to finish.")

print("\nScript execution finished.")